# Week 3 Project (Capstone)

## Import Libraries

In [46]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

## Create Pandas Data Frame

In [47]:
url = 'https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Toronto'
dfs = pd.read_html(url)

## Find Correct Table

In [48]:
hoods = dfs[10]
hoods

,CDN number,City-designated neighbourhood,Former city/borough,Neighbourhoods covered,Map,Unnamed: 5
0,129,Agincourt North,Scarborough,Agincourt and Brimwood,NaN,NaN
1,128,Agincourt South-Malvern West,Scarborough,Agincourt and Malvern,NaN,NaN
2,20,Alderwood,Etobicoke,Alderwood,NaN,NaN
3,95,Annex,Old City of Toronto,The Annex and Seaton Village,NaN,NaN
4,42,Banbury-Don Mills,North York,Don Mills,NaN,NaN
...,...,...,...,...,...,...
135,94,Wychwood,Old City of Toronto,NaN,NaN,NaN
136,100,Yonge and Eglinton,Old City of Toronto,Chaplin Estates,NaN,NaN
137,97,Yonge-St.Clair,Old City of Toronto,NaN,NaN,NaN
138,27,York University Heights,North York,NaN,NaN,NaN


## Drop Non-Required Information

In [52]:
#hoods = hoods.drop(['Neighbourhoods covered','Map', 'Unnamed: 5'], axis = 1)
hoods

,CDN number,City-designated neighbourhood,Former city/borough
0,129,Agincourt North,Scarborough
1,128,Agincourt South-Malvern West,Scarborough
2,20,Alderwood,Etobicoke
3,95,Annex,Old City of Toronto
4,42,Banbury-Don Mills,North York
...,...,...,...
135,94,Wychwood,Old City of Toronto
136,100,Yonge and Eglinton,Old City of Toronto
137,97,Yonge-St.Clair,Old City of Toronto
138,27,York University Heights,North York


In [50]:
hoods.shape

(140, 3)